In [3]:
## remember to run conn_cocolab from the terminal before running cells in this notebook!

import os
import urllib, cStringIO

import pymongo as pm

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')

import numpy as np
from __future__ import division
import scipy.stats as stats
import pandas as pd
import json
import re

from PIL import Image
import base64
import datetime

### file hierarchy and database connection vars

In [4]:
# set vars 
auth = pd.read_csv('../auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['cdm_run_v4']

### Preprocessing data and rendering images

Notes:
There are two types of records in the database: 'stroke' and 'finalImage'. This is stored under the key: 'dataType'.
The 'stroke'-type of record contains the svg string information. Every stroke event is stored as a separate record.
The session identifier is called "sessionId".




In [5]:
iterationName = 'cdm_run_v4'
all_sessions = coll.distinct('sessionId') ## this returns ALL sessions in this collection. we will then filter on time_threshold
image_recs_all = coll.find({'$and': [{'dataType':'finalImage'}]})
numImages = image_recs_all.count()
lastImage = image_recs_all[numImages - 1]
lastestDate = lastImage['date']

print 'We currently have {} valid sessions from {}.'.format(len(all_sessions), iterationName)
print 'We currently have {} total drawings from {}.'.format(numImages, iterationName)
print 'The last drawing was made at {}.'.format(lastestDate)

We currently have 1254 valid sessions from cdm_run_v4.
We currently have 8537 total drawings from cdm_run_v4.
The last drawing was made at 2018-11-12T23:16:00.927Z.


In [6]:
experiment_name = 'testing'
iterationName = 'cdm_run_v4'
all_sessions = coll.distinct('sessionId') ## this returns ALL sessions in this collection. we will then filter on time_threshold
valid_sessions = coll.find().distinct('sessionId')
image_recs_all = coll.find({'$and': [{'dataType':'finalImage'}]})
numImages = image_recs_all.count()
lastImage = image_recs_all[numImages - 1]
lastestDate = lastImage['date']

print 'We currently have {} valid sessions from {}.'.format(len(valid_sessions), iterationName)
print 'We currently have {} total drawings from {}.'.format(numImages, iterationName)
print 'The last drawing was made at {}.'.format(lastestDate)

We currently have 1254 valid sessions from cdm_run_v4.
We currently have 8537 total drawings from cdm_run_v4.
The last drawing was made at 2018-11-12T23:16:00.927Z.


In [28]:
coll = db['cdm_run_v3']
iterationName = 'cdm_run_v3'
all_sessions = coll.distinct('sessionId') ## this returns ALL sessions in this collection. we will then filter on time_threshold
image_recs_all = coll.find({'$and': [{'dataType':'finalImage'}]})
numImages = image_recs_all.count()
lastImage = image_recs_all[numImages - 1]
lastestDate = lastImage['date']

print 'We currently have {} valid sessions from {}.'.format(len(all_sessions), iterationName)
print 'We currently have {} total drawings from {}.'.format(numImages, iterationName)
print 'The last drawing was made at {}.'.format(lastestDate)

We currently have 4412 valid sessions from cdm_run_v3.
We currently have 25160 total drawings from cdm_run_v3.
The last drawing was made at 2018-09-04T21:33:25.905Z.


### How many of these drawings are from kids and not on practice trials?

In [6]:
image_recs_kids = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}},{'category':'square'}]})
print 'We currently have {} square images from kids in {}.'.format(image_recs_kids.count(), iterationName)

image_recs_kids = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}},{'category':'shape'}]})
print 'We currently have {} shape images from kids in {}.'.format(image_recs_kids.count(), iterationName)

image_recs_kids = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}},{'category':'this square'}]})
print 'We currently have {} circle images from kids in {}.'.format(image_recs_kids.count(), iterationName)

image_recs_kids = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}}]})
print 'We currently have {} TOTAL images from kids in {}.'.format(image_recs_kids.count(), iterationName)


We currently have 1158 square images from kids in cdm_run_v4.
We currently have 959 shape images from kids in cdm_run_v4.
We currently have 855 circle images from kids in cdm_run_v4.
We currently have 7836 TOTAL images from kids in cdm_run_v4.


In [7]:
image_recs_kids_not_practice = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}},{'category':{"$nin": ['square','this circle','shape']}}]})
print 'We currently have {} non-practice trials from kids in {}.'.format(image_recs_kids_not_practice.count(), iterationName)

We currently have 5842 non-practice trials from kids in cdm_run_v4.


In [8]:
image_recs_adults_not_practice = coll.find({'$and': [{'dataType':'finalImage'},{'age': 'adult'},{'category':{"$nin": ['square','this circle','shape']}}]})
print 'We currently have {} non-practice trials from adults in {}.'.format(image_recs_adults_not_practice.count(), iterationName)

We currently have 408 non-practice trials from adults in cdm_run_v4.


### Get out number of drawings per age and class

In [9]:
ages = {'age2','age3','age4','age5','age6','age7','age8','age9','age10'}
categories_run3 = {'a bear','a cat','a frog','a sheep','a key','a phone','a scissors','a train'}

In [22]:
# categories_run1 = {
# 'a boat',
# 'a car',
# 'a cup',
# 'a dog',
# 'a fish',
# 'a house',
# 'a person',
# 'a tree'}

# categories_run2 = {'an airplane',
# 'a bike',
# 'a bird',
# 'a bowl',
# 'a chair',
# 'a couch',
# 'a phone',
# 'a rabbit'}

# categories = {
# 'an airplane',
# 'a bike',
# 'a bird',
# 'a bowl',
# 'a chair',
# 'a couch',
# 'a phone',
# 'a rabbit',
# 'a boat',
# 'a car',
# 'a cup',
# 'a dog',
# 'a fish',
# 'a house',
# 'a person',
# 'a tree'}


In [ ]:
## Counts by age and clas
counts=[]
category=[]
age=[]
categories = categories_run3

for thiscat in categories:
    for thisage in ages:
        thissubset = coll.find({'$and': [{'dataType':'finalImage'},{'age':thisage},{'category':thiscat}]})
#         print 'We have {} drawings for {} and {} '.format(thissubset.count(),thisage, thiscat)
        counts.append(thissubset.count())
        category.append(thiscat)
        age.append(thisage)
        
C = pd.DataFrame([counts,category,age])
C = C.transpose()
C.columns = ['counts','category','age']

In [50]:
# for thisage in ages:
#     count_survey = coll.find({'$and': [{'dataType':'survey'},{'age':thisage}]})
#     print 'had {} surveys in {}'.format(count_survey.count(),thisage)
#     child_drew_survey = coll.find({'$and': [{'dataType':'survey'},{'age':thisage},{'child_drew':True}]})
#     other_drew_survey = coll.find({'$and': [{'dataType':'survey'},{'age':thisage},{'other_drew':True}]})
#     parent_drew_survey = coll.find({'$and': [{'dataType':'survey'},{'age':thisage},{'parent_drew':True}]})

#     print 'had {} surveys where kids drew in {}'.format(child_drew_survey.count(),thisage)
#     print 'had {} surveys where parents drew in {}'.format(parent_drew_survey.count(),thisage)
#     print 'had {} surveys where other kids drew in {}'.format(other_drew_survey.count(),thisage)


In [ ]:
# # Counts by class only
# cat_counts=[]
# for thiscat in categories:
#     thissubset = coll.find({'$and': [{'dataType':'finalImage'},{'age':{"$ne": 'adult'}},{'category':thiscat}]})
#     cat_counts.append(thissubset.count())

### Make some basic plots

In [52]:
# df.loc[df['column_name'] == some_value]
# bike = C.loc[C['category']=="a cat"]
# plt.figure(figsize=(14,4))
# plt.subplot(1,3,1)
# g=sns.barplot(data=bike,x='age',y='counts')
# for item in g.get_xticklabels():
#     item.set_rotation(45)
    
    
# couch = C.loc[C['category']=="a frog"]
# plt.figure(figsize=(14,4))
# plt.subplot(1,3,1)
# g=sns.barplot(data=couch,x='age',y='counts')
# for item in g.get_xticklabels():
#     item.set_rotation(45)

### Render out images from museum station -- takes a *long* time.

#### Grab some subset of desired sketches to render

In [49]:
cdm_run_v4 = db['cdm_run_v4']
cdm_run_v3 = db['cdm_run_v3']
this_collection = cdm_run_v4
##
sessions_to_render =  this_collection.find({'$and': [{'dataType':'finalImage'},{'age':'age5'}]}).distinct('sessionId')
print('we have {} unique kids'.format(len(sessions_to_render)))


we have 116 unique kids


#### And where are we rendering these skeches?

In [125]:
## And where are we rendering these sketches?
analysis_dir = os.getcwd()
sketch_dir = os.path.join(analysis_dir,'sketches_subset')
if not os.path.exists(sketch_dir):
    os.makedirs(sketch_dir)

#### Open up variables for csv file writing

In [130]:
# basic descriptors
session_id = []; trial_num = []; category = []; age = []; filename = []

# stroke times and duration
svg = []; svg_times = []; draw_duration_old = []; draw_duration_new = []

# drawing usage stats
num_strokes = []
mean_intensity = []
bounding_box = []

# trial time and duration
start_time = []; submit_time = []; trial_duration = []

# other timing variables
submit_date = []
submit_date_readable = []

def load_image_data(imgData,imsize):
    filestr = base64.b64decode(imgData)
    fname = os.path.join('sketch.png')
    with open(fname, "wb") as fh:
        fh.write(imgData.decode('base64'))
    im = Image.open(fname).resize((imsize,imsize))
    _im = np.array(im)
    return(_im)

def get_mean_intensity(img,imsize):
    thresh = 250
    numpix = imsize**2
    mean_intensity = len(np.where(img[:,:,3].flatten()>thresh)[0])/numpix
    return mean_intensity
    
def get_bounding_box(img):
    rows = np.any(img, axis=1)
    cols = np.any(img, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]
    bounding_box = tuple((rmin, rmax, cmin, cmax))
    return bounding_box

#### Render out sketches and save descriptives for subset

In [59]:
really_run_this=1

# basic variables for counting throughout the loop
skipCount = 0;
writeImageCount = 0;
interferenceCount = 0;
timeSave = []
imsize = 224 ## img writing size, optimized for CNN

if really_run_this:
   
    for s in sessions_to_render:        
        if (this_collection == cdm_run_v4):
            image_recs = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'finalImage'}]}).sort('startTrialTime')    
            
            ## get survey data for this kid (if it exists), use to exclude
            survey_session = this_collection.find({'$and': [{'dataType':'survey'},{'sessionId':s}]})
            if survey_session.count()>0:
                interference = (survey_session[0]['other_drew']==True | survey_session[0]['parent_drew']==True)
            else:
                interference = False
            
            if interference==True:
                interferenceCount = interferenceCount+1
                if np.mod(interferenceCount,10)==0:
                    print('excluded {} kids for reported inference...'.format(interferenceCount))

        elif (this_collection == cdm_run_v3):
            image_recs = this_collection.find({'$and': [{'sessionId':s}, {'dataType':'finalImage'}]}).sort('time')    
            interference = False

    if image_recs.count()>3 & interference==False: ## if they made it past the practice trials & no reported interference
            for imrec in image_recs:                                                            
                category_dir = os.path.join(sketch_dir,imrec['category'])
                if not os.path.exists(category_dir):
                    os.makedirs(category_dir)
                fname = os.path.join(category_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], imrec['age'],imrec['sessionId']))

                ## if this image exists already, skip it
                if os.path.isfile(fname):
                    skipCount = skipCount + 1;
                    if np.mod(skipCount,100)==0:
                        print('Weve skipped {} images...'.format(skipCount))
                        if (this_collection == cdm_run_v4):
                            timeSave.append(imrec['startTrialTime']) 
                        elif (this_collection == cdm_run_v3):
                            timeSave.append(imrec['time']) 
                else:
                    ## timing info was different in different collections, switch here
                    if (this_collection == cdm_run_v4):
                        stroke_recs = this_collection.find({'$and': [{'sessionId':s}, 
                                          {'dataType':'stroke'},
                                          {'trialNum': imrec['trialNum']}]}).sort('startTrialTime')   
                    elif (this_collection == cdm_run_v3):
                        stroke_recs = this_collection.find({'$and': [{'sessionId':s}, 
                                          {'dataType':'stroke'},
                                          {'trialNum': imrec['trialNum']}]}).sort('time')  

                    # don't do adults for now or blank images
                    if stroke_recs.count()>0 and imrec['age']!='adult': 
                        
                        ## Append session ID, trial Num, category, age                            
                        session_id.append(imrec['sessionId'])        
                        trial_num.append(imrec['trialNum']) 
                        category.append(imrec['category'])
                        age.append(imrec['age'])
                        
                        ## again, regularize based on timing info change
                        if (this_collection == cdm_run_v4):
                            start_time.append(imrec['startTrialTime'])
                            submit_time.append(imrec['endTrialTime'])
                            trial_duration.append((imrec['endTrialTime'] - imrec['startTrialTime'])/1000.00)
                            readadble_date = datetime.datetime.fromtimestamp(imrec['endTrialTime']/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')

                        elif (this_collection == cdm_run_v3):
                            start_time.append('NaN')
                            submit_time.append(imrec['time'])
                            trial_duration.append('NaN')
                            readadble_date = datetime.datetime.fromtimestamp(imrec['time']/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')

                        ## readable date (not just time, has other info for sanity cecks)
                        submit_date_readable.append(readadble_date)
                        submit_date.append(imrec['date'])
                        
                        ## Count number of strokes and timing information as well on stroke basis
                        num_strokes.append(stroke_recs.count())
                        _svg = [] # this keeps track of the strokes from THIS final image
                        _svg_end_times = []
                        _svg_start_times = []
                        for strec in stroke_recs:
                            _svg.append(strec['svg'])
                            _svg_end_times.append(strec['endStrokeTime'])
                            _svg_start_times.append(strec['startStrokeTime'])
                        ## draw duration
                        draw_duration_new.append((_svg_end_times[-1] - _svg_start_times[0])/1000) ## in seconds
                        draw_duration_old.append((_svg_end_times[-1] - _svg_end_times[0])/1000) ## in seconds
                        svg.append(_svg)
                        svg_times.append(_svg_times)
                        
                        ## get bounding box and mean pixel intensity
                        this_image = load_image_data(imrec['imgData'],imsize)
                        this_bounding_box = get_bounding_box(this_image)
                        this_intensity = get_mean_intensity(this_image,imsize)
                        #
                        bounding_box.append(this_bounding_box)
                        mean_intensity.append(this_intensity)
                        
                        ## Write out image data
                        imgData = imrec['imgData'];
                        filename.append(os.path.join(sketch_dir,'{}_sketch_{}_{}.png'.format(imrec['category'], imrec['age'],imrec['sessionId'])))
                        writeImageCount = writeImageCount+1
                        
                        with open(fname, "wb") as fh:
                            fh.write(imgData.decode('base64'))   
                        if np.mod(writeImageCount,10)==0:
                            if (this_collection == cdm_run_v4):
                                print('Weve written {} images at ... {} '.format(writeImageCount, imrec['startTrialTime']))
                            elif (this_collection == cdm_run_v3):
                                print('Weve written {} images at ... {} '.format(writeImageCount, imrec['time']))


NameError: name 'sketch_dir' is not defined

In [53]:
saveTime ='111'
if really_run_this:
    X_out = pd.DataFrame([sessionId,trial_num,category,age,submit_time,submit_date,num_strokes,draw_duration_old,draw_duration_new,trial_duration, mean_intensity, bounding_box, filename])
    X_out = X_out.transpose()
    X_out.columns = ['session_id','trial_num','category','age','submit_time','submit_date','num_strokes','draw_duration_old','draw_duration_new','trial_duration','mean_intensity','bounding_box','filename']
    X_out.to_csv('MuseumStation_AllDescriptives_{}.csv'.format(saveTime))

NameError: name 'sessionId' is not defined

In [ ]:
### Notes: It won't write out an image if it already exists. 
# Every time I get through a certain set of the  images, I record the time threshold 
#(since images are written out in chronological order)

## Time stamps every 100 images.
# [1525899407923.0,
#  1525980096385.0,
#  1526061840113.0,
#  1526072921014.0,
#  1526152365279.0,
#  1526160570739.0,
#  1526239856392.0,
#  1526251698552.0,
#  1526254954475.0,
#  1526418747635.0,
#  1526495927262.0,
#  1526503859627.0,
#  1526581190700.0,
#  1526593428928.0,
#  1526666014298.0,
#  1526680293892.0,
#  1526760285554.0,
#  1526766056162.0,
#  1527018277044.0,
#  1527793231321.0,
#  1527881543709.0,
#  1527960926822.0,
#  1527970125532.0,
#  1527977827733.0,
#  1527982709894.0,
#  1528059661642.0,
#  1528064507404.0,
#  1528068114827.0,
#  1528140581078.0,
#  1528225691857.0,
#  1528238270169.0,
#  1528321410290.0,
#  1528396097251.0,
#  1528409134891.0,
#  1528482501089.0,
#  1528492105365.0,
#  1528501366255.0,
#  1528579092501.0,
#  1528584985113.0,
#  1528661341497.0,
#  1528741749248.0,
#  1528747687974.0,
#  1528761045959.0,
#  1528835768499.0,
#  1528841240592.0,
#  1528916818096.0,
#  1528923498102.0,
#  1529005606993.0,
#  1529015740138.0,
#  1529092461367.0,
#  1529102005265.0,
#  1529178354515.0,
#  1529183429730.0,
#  1529190138056.0,
#  1529272790915.0,
#  1529351265769.0,
#  1529360098520.0,
#  1529435563121.0,
#  1529443877705.0,
#  1529518216924.0,
#  1529529413026.0,
#  1529690851878.0,
#  1529705056613.0,
#  1530311006680.0]


# time_threshold = 1530650649659.0 ## first ~7000 images! (cdm_run_v3)
# time_threshold = 1533252327962.0 ## all images up until morning of Aug 3 (cdm_run_v3)